In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
df = pd.read_excel('..\Dataset\KAROUSOU_ET_ECONOMACOU_EMERGENT_LITERACY_PREDICTION_OSF.xlsx')

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

Print all the important information about the data

In [ ]:
df.info()

Our target feature we want to predict is the LITERACY_3CLUSTERS. Here we try to examin the balance of our dataset

In [ ]:
df['LITERACY_3CLUSTERS'].value_counts()

In [ ]:
df_value_counts = df['LITERACY_3CLUSTERS'].value_counts()
ax = df_value_counts.plot(kind='pie', labels=['Litteracy 2', 'Literacy 1', 'Literacy 3'])
ax.set_ylabel('')

plt.savefig(r"..\Results\pie_graph_unbalancced_dataset.pdf", format="pdf", bbox_inches="tight")
plt.show()

We change the names of some columns. This happened to support the workflow of our paper. Also we group our features to 3 groups. The group A, group B and group C.

In [ ]:
# GROUP A
df.rename(columns={'PERCENT_NONVOCAL': 'NVOC'}, inplace=True)
df.rename(columns={'PERCENT_VOCAL': 'VOC'}, inplace=True)
df.rename(columns={'PERCENT_COMPR': 'COMPR'}, inplace=True)
df.rename(columns={'PERCENT_VOCAB': 'VOCAB'}, inplace=True)
df.rename(columns={'PERCENT_MORPHOL': 'MOR'}, inplace=True)
df.rename(columns={'PERCENT_SYNTAX': 'SYNT'}, inplace=True)

# GROUP B
df.rename(columns={'TOTAL_HLE': 'HLE'}, inplace=True)
df.rename(columns={'TOTAL_ISBR': 'ISBR'}, inplace=True)
df.rename(columns={'EDUCATION_1': 'EDU_M'}, inplace=True)
df.rename(columns={'HOURSMOTHER': 'HRS_M'}, inplace=True)
df.rename(columns={'hoursworkmother': 'HRSWORK_M'}, inplace=True)
df.rename(columns={'WORKSTATUS_1': 'WSTATUS_M'}, inplace=True)
df.rename(columns={'NR_CHILDREN': 'NRCHILD'}, inplace=True)
df.rename(columns={'URBANITY': 'URB'}, inplace=True)
df.rename(columns={'NRBOOKS': 'NRBOOKS'}, inplace=True)
df.rename(columns={'ECONOMIC': 'ECONOMIC'}, inplace=True)
df.rename(columns={'EDUCATION_2': 'EDU_F'}, inplace=True)
df.rename(columns={'HOURSFATHER': 'HRS_F'}, inplace=True)
df.rename(columns={'hoursworkfather': 'HRSWORK_F'}, inplace=True)
df.rename(columns={'WORKSTATUS_2': 'WSTATUS_F'}, inplace=True)
df.rename(columns={'BIRTHORDER': 'BIRTHORDER'}, inplace=True)

# GROUP C
df.rename(columns={'GENDER': 'SEX'}, inplace=True)
df.rename(columns={'BIRTHWEIGHT': 'BIRTHWEIGHT'}, inplace=True)
df.rename(columns={'PREMAWEEK': 'GESTWEEK'}, inplace=True)

In [ ]:
group_A = ['NVOC', 'VOC', 'COMPR', 'VOCAB', 'MOR', 'SYNT']
group_B = ['HLE', 'ISBR', 'EDU_M', 'HRS_M', 'HRSWORK_M', 'WSTATUS_M', 'NRCHILD', 'URB', 'NRBOOKS', 'ECONOMIC', 'EDU_F', 'HRS_F', 'HRSWORK_F', 'WSTATUS_F', 'BIRTHORDER']
group_C = ['BIRTHWEIGHT', 'GESTWEEK', 'SEX']

Define the columns from this dataset to use for the training. In each experiment we define a different subset of the Features. So change the next cell to choose different features.

In [ ]:
# This is the subset that predicts the best results.
df = df[['AGE'] + group_A + ['HRS_M', 'HRS_F', 'HRSWORK_M'] + ['LITERACY_3CLUSTERS']] 
print(df.info())

Use the pycaret to train and assess instantly 13 models. This give us the chance to see in each experiment the best model.

In [ ]:
from pycaret.classification import *
from sklearn.metrics import classification_report

# Step 1: Setup / We se a session_id=123 which is the random seep to ensure that our experiments is reproducible
clf = setup(data=df, target='LITERACY_3CLUSTERS', train_size=0.75, session_id=123, verbose=False, normalize=True)

# Step 2: Train and pick best model
best_model = compare_models()

# Step 3: Evaluate on the holdout set (25% of data)
predictions = predict_model(best_model)

from sklearn.metrics import classification_report
print(classification_report(predictions['LITERACY_3CLUSTERS'], predictions['prediction_label']))

The pycaret performs an automated data preprocessing. So we want to see the best model's hyperparameters and the pipeline of data preprocessing.

In [ ]:
print(best_model)

In [ ]:
print(best_model.get_params())

In [ ]:
get_config('pipeline') # The output is the data preprocessing workflow.

Compute Partial Dependency Plot for Age

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# Define number of classes and feature to plot
n_classes = 3
feature = "AGE"
class_labels = ["Class 0", "Class 1", "Class 2"]

# Get transformed training data
X_train_transformed = get_config('X_train_transformed')

# Create subplots
fig, axs = plt.subplots(1, n_classes, figsize=(5 * n_classes, 4), constrained_layout=True)

# Generate PDPs and add titles using display.axes_
for class_idx in range(n_classes):
    display = PartialDependenceDisplay.from_estimator(
        estimator=best_model,
        X=X_train_transformed,
        features=[feature],
        target=class_idx,
        grid_resolution=50,
        ax=axs[class_idx]
    )
    
    # Overwrite title using display.axes_
    display.axes_[0, 0].set_title(f"Partial Dependence – {class_labels[class_idx]}")
    display.axes_[0, 0].set_xlabel("AGE")
    display.axes_[0, 0].set_ylabel("Predicted Probability")

# Save to PDF
plt.savefig("..\Results\Partial_Dependence_AGE_T2_All_Classes.pdf", format="pdf", bbox_inches="tight")


Compute SHAP for Class 0 and visualize them.

In [ ]:
import shap
import matplotlib.pyplot as plt

max_display=10

# Get transformed training data
X_train_transformed = get_config('X_train_transformed')

# Create SHAP explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train_transformed)

import numpy as np

# Convert the list into a single NumPy array
shap_values_array = np.stack(shap_values)  # Now should be (171, 22, 3)

# Choose a class index (e.g., class 1)
class_index = 0
shap_values_class = shap_values_array[:, :, class_index]  # (171, 22)

# Now plot
shap.summary_plot(shap_values_class, X_train_transformed, plot_type="dot", max_display=max_display, show=False)
# Grab the current figure and save
plt.savefig("..\Results\Shapley_Value_Best_Model_Class_0.pdf", format="pdf", bbox_inches="tight")


ModuleNotFoundError: No module named 'shap'

Compute SHAP for Class 1 and visualize them.

In [ ]:
import shap
import matplotlib.pyplot as plt

max_display=10

# Get transformed training data
X_train_transformed = get_config('X_train_transformed')

# Create SHAP explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train_transformed)

import numpy as np

# Convert the list into a single NumPy array
shap_values_array = np.stack(shap_values)  # Now should be (171, 22, 3)

# Choose a class index (e.g., class 1)
class_index = 1
shap_values_class = shap_values_array[:, :, class_index]  # (171, 22)

# Now plot
shap.summary_plot(shap_values_class, X_train_transformed, plot_type="dot", max_display=max_display, show=False)
# Grab the current figure and save
plt.savefig("..\Results\Shapley_Value_Best_Model_Class_1.pdf", format="pdf", bbox_inches="tight")


Compute SHAP for Class 2 and visualize them.

In [ ]:
import shap
import matplotlib.pyplot as plt

max_display=10

# Get transformed training data
X_train_transformed = get_config('X_train_transformed')

# Create SHAP explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train_transformed)

import numpy as np

# Convert the list into a single NumPy array
shap_values_array = np.stack(shap_values)  # Now should be (171, 22, 3)

# Choose a class index (e.g., class 1)
class_index = 2
shap_values_class = shap_values_array[:, :, class_index]  # (171, 22)

# Now plot
shap.summary_plot(shap_values_class, X_train_transformed, plot_type="dot", max_display=max_display, show=False)
# Grab the current figure and save
plt.savefig("..\Results\Shapley_Value_Best_Model_Class_2.pdf", format="pdf", bbox_inches="tight")
